In [75]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:', echo=True)

Session = sessionmaker(bind=engine)

session = Session()

from datetime import datetime

from sqlalchemy import (Table, Column, Integer, Numeric, String, DateTime,
                        ForeignKey, Boolean)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class Cookie(Base):
    __tablename__ = 'cookies'

    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))

    def __init__(self, name, recipe_url=None, sku=None, quantity=0, unit_cost=0.00):
        self.cookie_name = name
        self.cookie_recipe_url = recipe_url
        self.cookie_sku = sku
        self.quantity = quantity
        self.unit_cost = unit_cost

    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}' ,"\
                        "cookie_recipe_url='{self.cookie_recipe_url}', " \
                        "cookie_sku='{self.cookie_sku}', "\
                        "quantity={self.quantity}, "\
                        "unit_cost={self.unit_cost}".format(self=self)

class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __init__(self, username, email_address, phone, password):
        self.username = username
        self.email_address = email_address
        self.phone = phone
        self.password = password

    def __repr__(self):
        return "User(username='{self.username}', " \
                    "email_address='{self.email_address}', "\
                    "phone='{self.phone}', " \
                    "password='{self.password}')".format(self=self)


class Order(Base):
    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)

    user = relationship("User", backref=backref('order', order_by=order_id))

    def __repr__(self):
        return "Order(user_id={self.user_id}, "\
                "shipped={self.shipped})".format(self=self)

class LineItem(Base):
    __tablename__ = 'line_items'
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))

    order = relationship("Order", backref=backref("line_items", order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False)

    def __repr__(self):
        return "LineItems(order_id={self.order_id}, "\
                            "cookie_id={self.cookie_id}, "\
                            "quantity={self.quantity}, "\
                            "extended_cost={self.extended_cost}".format(self=self)


Base.metadata.create_all(engine)



2024-01-03 16:29:02,807 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 16:29:02,810 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2024-01-03 16:29:02,812 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 16:29:02,814 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cookies")
2024-01-03 16:29:02,815 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 16:29:02,817 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-01-03 16:29:02,818 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 16:29:02,820 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2024-01-03 16:29:02,821 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 16:29:02,821 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-01-03 16:29:02,822 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 16:29:02,822 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2024-01-03 16:29:02,823 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-

In [76]:
cc_cookie = Cookie('chocolate chip',
                   'http://some.aweso.me/cookie/recipe.html',
                   'CC01', 12, 0.50)

In [77]:
from sqlalchemy import inspect
insp = inspect(cc_cookie)

In [78]:
for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: True
   pending: False
persistent: False
  detached: False


In [79]:
session.add(cc_cookie)
cc_cookie.cookie_name = 'Change chocolate chip'

In [89]:
insp.modified

False

In [88]:
for attr, attr_state in insp.attrs.items():
    if attr_state.history.has_changes():
        print('{}: {}'.format(attr, attr_state.value))
        print('History: {}\n'.format(attr_state.history))

In [82]:
dcc = Cookie('dark chocolate chip',
             'http://some.aweso.me/cookie/recipe_dark.html',
             'CC02', 1, 0.75)

session.add(dcc)
session.commit()

2024-01-03 16:29:03,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 16:29:03,059 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 16:29:03,061 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('Change chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, 0.5)
2024-01-03 16:29:03,063 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 16:29:03,064 INFO sqlalchemy.engine.Engine [cached since 0.005599s ago] ('dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75)
2024-01-03 16:29:03,066 INFO sqlalchemy.engine.Engine COMMIT


In [83]:
for row in session.query(Cookie).all():
    print(row)

2024-01-03 16:29:03,080 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 16:29:03,083 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2024-01-03 16:29:03,084 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
Cookie(cookie_name='Change chocolate chip' ,cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=12, unit_cost=0.50
Cookie(cookie_name='dark chocolate chip' ,cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75


C:\Users\WX847\AppData\Local\Temp\ipykernel_142996\4015204126.py:1: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for row in session.query(Cookie).all():


In [84]:
# Example 8-3 Causing a MultipleResultsFound exception
results = session.query(Cookie).one()

2024-01-03 16:29:03,111 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2024-01-03 16:29:03,112 INFO sqlalchemy.engine.Engine [cached since 0.02873s ago] ()


MultipleResultsFound: Multiple rows were found when exactly one was required

In [85]:
from sqlalchemy.orm.exc import MultipleResultsFound
try:
    results = session.query(Cookie).one()
except MultipleResultsFound as error:
    print("We found to many cookies... is that even possible?")

2024-01-03 16:29:10,367 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2024-01-03 16:29:10,368 INFO sqlalchemy.engine.Engine [cached since 7.285s ago] ()
We found to many cookies... is that even possible?


In [86]:
# Example 8-6 Createing a user and an order for that user
cookiemon = User('cookiemon', 'mon@cookie.com', '111-111-1111','password')
session.add(cookiemon)
o1 = Order()
o1.user = cookiemon
session.add(o1)

cc = session.query(Cookie).filter(Cookie.cookie_name == "Change chocolate chip").one()

line1 = LineItem(order=o1, cookie=cc, quantity=2, extended_cost=1.00)

session.add(line1)
session.commit()

2024-01-03 16:29:12,386 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)
2024-01-03 16:29:12,388 INFO sqlalchemy.engine.Engine [generated in 0.00240s] ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2024-01-03 16:29:12.385114', '2024-01-03 16:29:12.385114')
2024-01-03 16:29:12,391 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, shipped) VALUES (?, ?)
2024-01-03 16:29:12,392 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (1, 0)
2024-01-03 16:29:12,395 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
2024-01-03 16:29:12,396 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ('C

In [91]:
insp = inspect(o1)

for state in ['transient', 'pending', 'persistent', 'detached']:
    print('{:>10}: {}'.format(state, getattr(insp, state)))

 transient: False
   pending: False
persistent: False
  detached: True


In [92]:
# Example 8-7 Causing a DetachedInstanceError
order = session.query(Order).first()
session.expunge(order)
order.line_items

2024-01-03 16:38:17,075 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.shipped AS orders_shipped 
FROM orders
 LIMIT ? OFFSET ?
2024-01-03 16:38:17,076 INFO sqlalchemy.engine.Engine [cached since 215.1s ago] (1, 0)


DetachedInstanceError: Parent instance <Order at 0x21aab2474f0> is not bound to a Session; lazy load operation of attribute 'line_items' cannot proceed (Background on this error at: https://sqlalche.me/e/14/bhk3)

In [93]:
order.order_id

1